In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from functools import reduce
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 150)
from functools import reduce
pd.options.mode.chained_assignment = None  # default='warn'
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
import sys
sys.path.append("../../Functions and Dictionaries/") # Adds higher directory to python modules path
import geodict
namestocommon = geodict.namestocommon
geotogeoid = geodict.geotogeoid
tofullcensus = geodict.tofullcensus
import sqlite3 as sq
#functions
def percent(x, y):
        try:
            return ((x/y)*100)
        except ZeroDivisionError:
            return 0
def percentchange(x, y):
    try:
        return ((x - y)*100/y)
    except ZeroDivisionError:
        return 0
def realchange(x, y):
    return x-y
#calculate real and percent change between all columns for all possible time frames
def calculate_changes(df, columns, time_frames, years):
    for column in columns:
        for time_frame in time_frames:
            start_year, end_year = time_frame.split('-')
            df[f'{column} % Change', 'None', f'{time_frame}'] = percentchange(df[(column, int(end_year), 'None')], df[(column, int(start_year), 'None')])
            df[f'{column} Change', 'None', f'{time_frame}'] = (df[(column, int(end_year), 'None')] - df[(column, int(start_year), 'None')])

    return df
#generate all possible time frames from a list of years
def generate_time_frames(years):
    time_frames = []
    for i in range(len(years)-1):
        for j in range(i+1, len(years)):
            time_frames.append(f"{years[i]}-{years[j]}")
    return time_frames

In [87]:
data = pd.read_csv('data/ecdreq.csv')

In [88]:
data.head()

,NAME,GEO_ID,Employment:Armed Forces,Employment:Not in Labor Force,Employment:Employed Civilian,Employment:Unemployed Civilian,Employment%:Civilian Population in LF 16 Over that are Employed,Employment%:Total Civilian Population 16 Over that are Employed,Employment%:Population 16 Over in LF that are Employed,Employment%:Population 16 Over that are Employed Civilians,Employment%:Civilian Population in LF 16 Over that are Unemployed,Employment%:Total Civilian Population 16 Over that are Unemployed,Employment%:Population 16 Over in LF that are Unemployed,Employment%:Population 16 Over that are Unemployed Civilians,Employment%:Civilian Population 16 Over Not in Labor Force,Employment%:Total Population 16 Over Not in Labor Force,Employment:Total Population 16 Over In Labor Force,Employment%:Total Population 16 Over In Labor Force,Employment:Civilian Population 16 Over In Labor Force,Employment%:Civilian Population 16 Over In Labor Force,Employment%:Population 16 Over in LF Employed in Armed Forces,Employment%:Population 16 Over Employed in Armed Forces,Employment:Armed Forces American Indian Alaska Native Alone,Employment:Not in Labor Force American Indian Alaska Native Alone,Employment:Employed Civilian American Indian Alaska Native Alone,Employment:Unemployed Civilian American Indian Alaska Native Alone,Employment%:Civilian Population in LF 16 Over that are Employed American Indian Alaska Native Alone,Employment%:Total Civilian Population 16 Over that are Employed American Indian Alaska Native Alone,Employment%:Population 16 Over in LF that are Employed American Indian Alaska Native Alone,Employment%:Population 16 Over that are Employed Civilians American Indian Alaska Native Alone,Employment%:Civilian Population in LF 16 Over that are Unemployed American Indian Alaska Native Alone,Employment%:Total Civilian Population 16 Over that are Unemployed American Indian Alaska Native Alone,Employment%:Population 16 Over in LF that are Unemployed American Indian Alaska Native Alone,Employment%:Population 16 Over that are Unemployed Civilians American Indian Alaska Native Alone,Employment%:Civilian Population 16 Over Not in Labor Force American Indian Alaska Native Alone,Employment%:Total Population 16 Over Not in Labor Force American Indian Alaska Native Alone,Employment:Total Population 16 Over In Labor Force American Indian Alaska Native Alone,Employment%:Total Population 16 Over In Labor Force American Indian Alaska Native Alone,Employment:Civilian Population 16 Over In Labor Force American Indian Alaska Native Alone,Employment%:Civilian Population 16 Over In Labor Force American Indian Alaska Native Alone,Employment%:Population 16 Over in LF Employed in Armed Forces American Indian Alaska Native Alone,Employment%:Population 16 Over Employed in Armed Forces American Indian Alaska Native Alone,Employment:Armed Forces Native Hawaiian Pacific Islander Alone,Employment:Not in Labor Force Native Hawaiian Pacific Islander Alone,Employment:Employed Civilian Native Hawaiian Pacific Islander Alone,Employment:Unemployed Civilian Native Hawaiian Pacific Islander Alone,Employment%:Civilian Population in LF 16 Over that are Employed Native Hawaiian Pacific Islander Alone,Employment%:Total Civilian Population 16 Over that are Employed Native Hawaiian Pacific Islander Alone,Employment%:Population 16 Over in LF that are Employed Native Hawaiian Pacific Islander Alone,Employment%:Population 16 Over that are Employed Civilians Native Hawaiian Pacific Islander Alone,Employment%:Civilian Population in LF 16 Over that are Unemployed Native Hawaiian Pacific Islander Alone,Employment%:Total Civilian Population 16 Over that are Unemployed Native Hawaiian Pacific Islander Alone,Employment%:Population 16 Over in LF that are Unemployed Native Hawaiian Pacific Islander Alone,Employment%:Population 16 Over that are Unemployed Civilians Native Hawaiian Pacific Islander Alone,Employment%:Civilian Population 16 Over Not in Labor Force Native Hawaiian Pacific Islander

In [89]:
cols = ['Employment:Employed Civilian', 'Employment:Unemployed Civilian', 'Employment:Not in Labor Force', 
             'Employment:Employed Civilian American Indian Alaska Native Alone', 'Employment:Unemployed Civilian American Indian Alaska Native Alone', 
             'Employment:Not in Labor Force American Indian Alaska Native Alone', 'Employment:Employed Civilian Native Hawaiian Pacific Islander Alone', 
             'Employment:Unemployed Civilian Native Hawaiian Pacific Islander Alone', 'Employment:Not in Labor Force Native Hawaiian Pacific Islander Alone', 
             'Employment:Employed Civilian Non-Minority', 'Employment:Unemployed Civilian Non-Minority', 'Employment:Not in Labor Force Non-Minority'
            ]
data = data[cols]

In [90]:
data['Employment:Employed Civilian Group'] = data['Employment:Employed Civilian American Indian Alaska Native Alone'] + data['Employment:Employed Civilian Native Hawaiian Pacific Islander Alone']
data['Employment:Unemployed Civilian Group'] = data['Employment:Unemployed Civilian American Indian Alaska Native Alone'] + data['Employment:Unemployed Civilian Native Hawaiian Pacific Islander Alone']
data['Employment:Civilian Labor Force Group'] = data['Employment:Employed Civilian Group'] + data['Employment:Unemployed Civilian Group']
data['Civilian Unemployment Rate Group'] = (data['Employment:Unemployed Civilian Group']/data['Employment:Civilian Labor Force Group'])*100

In [91]:
data

,Employment:Employed Civilian,Employment:Unemployed Civilian,Employment:Not in Labor Force,Employment:Employed Civilian American Indian Alaska Native Alone,Employment:Unemployed Civilian American Indian Alaska Native Alone,Employment:Not in Labor Force American Indian Alaska Native Alone,Employment:Employed Civilian Native Hawaiian Pacific Islander Alone,Employment:Unemployed Civilian Native Hawaiian Pacific Islander Alone,Employment:Not in Labor Force Native Hawaiian Pacific Islander Alone,Employment:Employed Civilian Non-Minority,Employment:Unemployed Civilian Non-Minority,Employment:Not in Labor Force Non-Minority,Employment:Employed Civilian Group,Employment:Unemployed Civilian Group,Employment:Civilian Labor Force Group,Civilian Unemployment Rate Group
0,3201140.0,179568.0,2105502.0,5880.0,469.0,5368.0,2121.0,358.0,676.0,2377207.0,110233.0,1642496.0,8001.0,827.0,8828.0,9.367920
1,157510982.0,9161615.0,96218516.0,1109518.0,111867.0,856979.0,288797.0,22342.0,162769.0,96699083.0,4518905.0,61931932.0,1398315.0,134209.0,1532524.0,8.757383


In [92]:
data['Employment:Civilian Labor Force'] = data['Employment:Employed Civilian'] + data['Employment:Unemployed Civilian']
data['Civilian Unemployment Rate'] = (data['Employment:Unemployed Civilian']/data['Employment:Civilian Labor Force'])*100

In [93]:
data['Employment:Civilian Labor Force Non-Minority'] = data['Employment:Employed Civilian Non-Minority'] + data['Employment:Unemployed Civilian Non-Minority']
data['Civilian Unemployment Rate Non-Minority'] = (data['Employment:Unemployed Civilian Non-Minority']/data['Employment:Civilian Labor Force Non-Minority'])*100

In [94]:
#data = data.drop(columns = cols)

In [95]:
data.head()

,Employment:Employed Civilian,Employment:Unemployed Civilian,Employment:Not in Labor Force,Employment:Employed Civilian American Indian Alaska Native Alone,Employment:Unemployed Civilian American Indian Alaska Native Alone,Employment:Not in Labor Force American Indian Alaska Native Alone,Employment:Employed Civilian Native Hawaiian Pacific Islander Alone,Employment:Unemployed Civilian Native Hawaiian Pacific Islander Alone,Employment:Not in Labor Force Native Hawaiian Pacific Islander Alone,Employment:Employed Civilian Non-Minority,Employment:Unemployed Civilian Non-Minority,Employment:Not in Labor Force Non-Minority,Employment:Employed Civilian Group,Employment:Unemployed Civilian Group,Employment:Civilian Labor Force Group,Civilian Unemployment Rate Group,Employment:Civilian Labor Force,Civilian Unemployment Rate,Employment:Civilian Labor Force Non-Minority,Civilian Unemployment Rate Non-Minority
0,3201140.0,179568.0,2105502.0,5880.0,469.0,5368.0,2121.0,358.0,676.0,2377207.0,110233.0,1642496.0,8001.0,827.0,8828.0,9.367920,3380708.0,5.311550,2487440.0,4.431584
1,157510982.0,9161615.0,96218516.0,1109518.0,111867.0,856979.0,288797.0,22342.0,162769.0,96699083.0,4518905.0,61931932.0,1398315.0,134209.0,1532524.0,8.757383,166672597.0,5.496773,101217988.0,4.464528


In [39]:
data.to_csv('employmentbyrace.csv', index = False)

In [60]:
moredata = pd.read_csv('data/ecdreq1.csv')

In [61]:
moredata.head()

,Unnamed: 0,NAME,GEO_ID,pop,raceeth_whitealone_nothispanicorlatino,raceeth_hispanicorlatino,units_allhousing,occupancy_total_series,occupancy_occupiedunits,occupancy_vacantunits,gq_total_series,gq_institutionalized,gq_institutionalized_correctionalfacilityadults,gq_institutionalized_juvenilefacility,gq_institutionalized_nursingorskillednursingfacility,gq_institutionalized_other,gq_noninstitutionalized,gq_noninstitutionalized_collegeoruniversitystudenthousing,gq_noninstitutionalized_militaryquarters,gq_noninstitutionalized_other,raceeth_total_series,raceeth_white_alone,raceeth_blackafricanamerican_alone,raceeth_americanindianalaskanative_alone,raceeth_asian_alone,raceeth_nativehawaiianotherpacificislander_alone,raceeth_someotherrace_alone,raceeth_twoormoreraces,raceeth_tworaces,raceeth_tworaces_white_blackafricanamerican,raceeth_tworaces_white_americanindianalaskanative,raceeth_tworaces_white_asian,raceeth_tworaces_white_nativehawaiianpacificislander,raceeth_tworaces_white_someotherrace,raceeth_tworaces_blackafricanamerican_americanindianalaskanative,raceeth_tworaces_blackafricanamerican_asian,raceeth_tworaces_blackafricanamerican_nativehawaiianpacificislander,raceeth_tworaces_blackafricanamerican_someotherrace,raceeth_tworaces_americanindianalaskanative_asian,raceeth_tworaces_americanindianalaskanative_nativehawaiianpacificislander,raceeth_tworaces_americanindianalaskanative_someotherrace,raceeth_tworaces_asiannativehawaiianpacificislander,raceeth_tworaces_asiansomeotherrace,raceeth_tworaces_nativehawaiianotherpacificislandersomeotherrace,raceeth_threeraces,raceeth_threeraces_white_blackafricanamerican_americanindianalaskanative,raceeth_threeraces_white_blackafricanamerican_asian,raceeth_threeraces_white_blackafricanamerican_nativehawaiianpacificislander,raceeth_threeraces_white_blackafricanamerican_someotherrace,raceeth_threeraces_white_americanindianalaskanative_asian,raceeth_threeraces_white_americanindianalaskanative_nativehawaiianpacificislander,raceeth_threeraces_white_americanindianalaskanative_someotherrace,raceeth_threeraces_white_asian_nativehawaiianpacificislander,raceeth_threeraces_white_asian_someotherrace,raceeth_threeraces_white_nativehawaiianpacificislander_someotherrace,raceeth_threeraces_blackafricanamerican_americanindianalaskanative_asian,raceeth_threeraces_blackafricanamerican_americanindianalaskanative_nativehawaiianpacificislander,raceeth_threeraces_blackafricanamerican_americanindianalaskanative_someotherrace,raceeth_threeraces_blackafricanamerican_asian_nativehawaiianpacificislander,raceeth_threeraces_blackafricanamerican_asian_someotherrace,raceeth_threeraces_blackafricanamerican_nativehawaiianpacificislander_someotherrace,raceeth_threeraces_americanindianalaskanative_asian_nativehawaiianpacificislander,raceeth_threeraces_americanindianalaskanative_asian_someotherrace,raceeth_threeraces_americanindianalaskanative_nativehawaiianpacificislander_someotherrace,raceeth_threeraces_asian_nativehawaiianpacificislander_someotherrace,raceeth_fourraces,raceeth_fourraces_white_blackafricanamerican_americanindianalaskanative_asian,raceeth_fourraces_white_blackafricanamerican_americanindianalaskanative_nativehawaiianpacificislander,raceeth_fourraces_white_blackafricanamerican_americanindianalaskanative_someotherrace,raceeth_fourraces_white_blackafricanamerican_asian_nativehawaiianpacificislander,raceeth_fourraces_white_blackafricanamerican_asian_someotherrace,raceeth_fourraces_white_blackafricanamerican_nativehawaiianpacificislander_someotherrace,raceeth_fourraces_white_americanindianalaskanative_asian_nativehawaiianpacificislander,raceeth_fourraces_white_americanindianalaskanative_asian_someotherrace,raceeth_fourraces_white_americanindianalaskanative_nativehawaiianpacificislander_someotherrace,raceeth_fourraces_white_asian_nativehawaiianpacificislander_someotherrace,raceeth_fourraces_blackafricanamerican_americanindianalaskanative_asian_nativehawaiianpacificislander,raceeth_fourraces_blackafricanamerican_americani

In [62]:
  moredata = moredata.drop(columns = ['Unnamed: 0', 'NAME', 'GEO_ID'])

In [63]:
moredata.head()

,pop,raceeth_whitealone_nothispanicorlatino,raceeth_hispanicorlatino,units_allhousing,occupancy_total_series,occupancy_occupiedunits,occupancy_vacantunits,gq_total_series,gq_institutionalized,gq_institutionalized_correctionalfacilityadults,gq_institutionalized_juvenilefacility,gq_institutionalized_nursingorskillednursingfacility,gq_institutionalized_other,gq_noninstitutionalized,gq_noninstitutionalized_collegeoruniversitystudenthousing,gq_noninstitutionalized_militaryquarters,gq_noninstitutionalized_other,raceeth_total_series,raceeth_white_alone,raceeth_blackafricanamerican_alone,raceeth_americanindianalaskanative_alone,raceeth_asian_alone,raceeth_nativehawaiianotherpacificislander_alone,raceeth_someotherrace_alone,raceeth_twoormoreraces,raceeth_tworaces,raceeth_tworaces_white_blackafricanamerican,raceeth_tworaces_white_americanindianalaskanative,raceeth_tworaces_white_asian,raceeth_tworaces_white_nativehawaiianpacificislander,raceeth_tworaces_white_someotherrace,raceeth_tworaces_blackafricanamerican_americanindianalaskanative,raceeth_tworaces_blackafricanamerican_asian,raceeth_tworaces_blackafricanamerican_nativehawaiianpacificislander,raceeth_tworaces_blackafricanamerican_someotherrace,raceeth_tworaces_americanindianalaskanative_asian,raceeth_tworaces_americanindianalaskanative_nativehawaiianpacificislander,raceeth_tworaces_americanindianalaskanative_someotherrace,raceeth_tworaces_asiannativehawaiianpacificislander,raceeth_tworaces_asiansomeotherrace,raceeth_tworaces_nativehawaiianotherpacificislandersomeotherrace,raceeth_threeraces,raceeth_threeraces_white_blackafricanamerican_americanindianalaskanative,raceeth_threeraces_white_blackafricanamerican_asian,raceeth_threeraces_white_blackafricanamerican_nativehawaiianpacificislander,raceeth_threeraces_white_blackafricanamerican_someotherrace,raceeth_threeraces_white_americanindianalaskanative_asian,raceeth_threeraces_white_americanindianalaskanative_nativehawaiianpacificislander,raceeth_threeraces_white_americanindianalaskanative_someotherrace,raceeth_threeraces_white_asian_nativehawaiianpacificislander,raceeth_threeraces_white_asian_someotherrace,raceeth_threeraces_white_nativehawaiianpacificislander_someotherrace,raceeth_threeraces_blackafricanamerican_americanindianalaskanative_asian,raceeth_threeraces_blackafricanamerican_americanindianalaskanative_nativehawaiianpacificislander,raceeth_threeraces_blackafricanamerican_americanindianalaskanative_someotherrace,raceeth_threeraces_blackafricanamerican_asian_nativehawaiianpacificislander,raceeth_threeraces_blackafricanamerican_asian_someotherrace,raceeth_threeraces_blackafricanamerican_nativehawaiianpacificislander_someotherrace,raceeth_threeraces_americanindianalaskanative_asian_nativehawaiianpacificislander,raceeth_threeraces_americanindianalaskanative_asian_someotherrace,raceeth_threeraces_americanindianalaskanative_nativehawaiianpacificislander_someotherrace,raceeth_threeraces_asian_nativehawaiianpacificislander_someotherrace,raceeth_fourraces,raceeth_fourraces_white_blackafricanamerican_americanindianalaskanative_asian,raceeth_fourraces_white_blackafricanamerican_americanindianalaskanative_nativehawaiianpacificislander,raceeth_fourraces_white_blackafricanamerican_americanindianalaskanative_someotherrace,raceeth_fourraces_white_blackafricanamerican_asian_nativehawaiianpacificislander,raceeth_fourraces_white_blackafricanamerican_asian_someotherrace,raceeth_fourraces_white_blackafricanamerican_nativehawaiianpacificislander_someotherrace,raceeth_fourraces_white_americanindianalaskanative_asian_nativehawaiianpacificislander,raceeth_fourraces_white_americanindianalaskanative_asian_someotherrace,raceeth_fourraces_white_americanindianalaskanative_nativehawaiianpacificislander_someotherrace,raceeth_fourraces_white_asian_nativehawaiianpacificislander_someotherrace,raceeth_fourraces_blackafricanamerican_americanindianalaskanative_asian_nativehawaiianpacificislander,raceeth_fourraces_blackafricanamerican_americanindianalaskanative_asian

In [65]:
cols = ['pop', 
        'raceeth_americanindianalaskanative_alone', 
        'raceeth_nativehawaiianotherpacificislander_alone', 
        'raceeth_tworaces_white_americanindianalaskanative', 
        'raceeth_tworaces_white_nativehawaiianpacificislander', 
        'raceeth_tworaces_blackafricanamerican_americanindianalaskanative', 
        'raceeth_tworaces_blackafricanamerican_nativehawaiianpacificislander', 
        'raceeth_tworaces_americanindianalaskanative_asian', 
        'raceeth_tworaces_americanindianalaskanative_nativehawaiianpacificislander', 
        'raceeth_tworaces_americanindianalaskanative_someotherrace', 
        'raceeth_tworaces_asiannativehawaiianpacificislander', 
        'raceeth_tworaces_nativehawaiianotherpacificislandersomeotherrace', 
        'raceeth_threeraces_white_blackafricanamerican_americanindianalaskanative', 
       'raceeth_tworaces_americanindianalaskanative_someotherrace', 'raceeth_tworaces_asiannativehawaiianpacificislander', 
        'raceeth_tworaces_nativehawaiianotherpacificislandersomeotherrace', 'raceeth_threeraces_white_blackafricanamerican_americanindianalaskanative', 
        'raceeth_threeraces_white_blackafricanamerican_nativehawaiianpacificislander', 'raceeth_threeraces_white_americanindianalaskanative_asian', 
        'raceeth_threeraces_white_americanindianalaskanative_nativehawaiianpacificislander', 'raceeth_threeraces_white_americanindianalaskanative_someotherrace', 
        'raceeth_threeraces_white_asian_nativehawaiianpacificislander', 'raceeth_threeraces_white_asian_someotherrace', 
        'raceeth_threeraces_white_nativehawaiianpacificislander_someotherrace', 'raceeth_threeraces_blackafricanamerican_americanindianalaskanative_asian', 
        'raceeth_threeraces_blackafricanamerican_americanindianalaskanative_nativehawaiianpacificislander', 
        'raceeth_threeraces_blackafricanamerican_americanindianalaskanative_someotherrace', 
        'raceeth_threeraces_blackafricanamerican_asian_nativehawaiianpacificislander', 'raceeth_threeraces_blackafricanamerican_asian_someotherrace', 
        'raceeth_threeraces_blackafricanamerican_nativehawaiianpacificislander_someotherrace', 
        'raceeth_threeraces_americanindianalaskanative_asian_nativehawaiianpacificislander', 'raceeth_threeraces_americanindianalaskanative_asian_someotherrace', 
        'raceeth_threeraces_americanindianalaskanative_nativehawaiianpacificislander_someotherrace', 
        'raceeth_threeraces_asian_nativehawaiianpacificislander_someotherrace', 'raceeth_fourraces_white_blackafricanamerican_americanindianalaskanative_asian',
        'raceeth_fourraces_white_blackafricanamerican_americanindianalaskanative_nativehawaiianpacificislander', 
        'raceeth_fourraces_white_blackafricanamerican_americanindianalaskanative_someotherrace', 
        'raceeth_fourraces_white_blackafricanamerican_asian_nativehawaiianpacificislander',
        'raceeth_fourraces_white_blackafricanamerican_nativehawaiianpacificislander_someotherrace', 
        'raceeth_fourraces_white_americanindianalaskanative_asian_nativehawaiianpacificislander', 
        'raceeth_fourraces_white_americanindianalaskanative_asian_someotherrace', 
        'raceeth_fourraces_white_americanindianalaskanative_nativehawaiianpacificislander_someotherrace', 
        'raceeth_fourraces_white_asian_nativehawaiianpacificislander_someotherrace', 
        'raceeth_fourraces_blackafricanamerican_americanindianalaskanative_asian_nativehawaiianpacificislander', 
        'raceeth_fourraces_blackafricanamerican_americanindianalaskanative_asian_someotherrace', 
        'raceeth_fourraces_blackafricanamerican_americanindianalaskanative_nativehawaiianpacificislander_someotherrace', 
        'raceeth_fourraces_blackafricanamerican_asian_nativehawaiianpacificislander_someotherrace', 
        'raceeth_fourraces_americanindianalaskanative_asian_nativehawaiianpacificislander_someotherrace', 'raceeth_fiveraces', 'raceeth_sixraces'
       ]
moredata = moredata[cols]

In [67]:
ref = moredata[['pop', 
        'raceeth_americanindianalaskanative_alone', 
        'raceeth_nativehawaiianotherpacificislander_alone']]

In [81]:
somenative

0    173154.0
Name: all with some native, dtype: float64

In [83]:
totalpop = ref['pop']
nativealone = ref['raceeth_americanindianalaskanative_alone'] + ref['raceeth_nativehawaiianotherpacificislander_alone']

In [84]:
(nativealone/totalpop)*100

0    0.465341
dtype: float64

In [85]:
nativealone

0    32159.0
dtype: float64

In [86]:
totalpop

0    6910840.0
Name: pop, dtype: float64

In [69]:
moredata = moredata.drop(columns = ['pop', 
        'raceeth_americanindianalaskanative_alone', 
        'raceeth_nativehawaiianotherpacificislander_alone'])

In [70]:
moredata.head()

,raceeth_tworaces_white_americanindianalaskanative,raceeth_tworaces_white_nativehawaiianpacificislander,raceeth_tworaces_blackafricanamerican_americanindianalaskanative,raceeth_tworaces_blackafricanamerican_nativehawaiianpacificislander,raceeth_tworaces_americanindianalaskanative_asian,raceeth_tworaces_americanindianalaskanative_nativehawaiianpacificislander,raceeth_tworaces_americanindianalaskanative_someotherrace,raceeth_tworaces_asiannativehawaiianpacificislander,raceeth_tworaces_nativehawaiianotherpacificislandersomeotherrace,raceeth_threeraces_white_blackafricanamerican_americanindianalaskanative,raceeth_tworaces_americanindianalaskanative_someotherrace,raceeth_tworaces_asiannativehawaiianpacificislander,raceeth_tworaces_nativehawaiianotherpacificislandersomeotherrace,raceeth_threeraces_white_blackafricanamerican_americanindianalaskanative,raceeth_threeraces_white_blackafricanamerican_nativehawaiianpacificislander,raceeth_threeraces_white_americanindianalaskanative_asian,raceeth_threeraces_white_americanindianalaskanative_nativehawaiianpacificislander,raceeth_threeraces_white_americanindianalaskanative_someotherrace,raceeth_threeraces_white_asian_nativehawaiianpacificislander,raceeth_threeraces_white_asian_someotherrace,raceeth_threeraces_white_nativehawaiianpacificislander_someotherrace,raceeth_threeraces_blackafricanamerican_americanindianalaskanative_asian,raceeth_threeraces_blackafricanamerican_americanindianalaskanative_nativehawaiianpacificislander,raceeth_threeraces_blackafricanamerican_americanindianalaskanative_someotherrace,raceeth_threeraces_blackafricanamerican_asian_nativehawaiianpacificislander,raceeth_threeraces_blackafricanamerican_asian_someotherrace,raceeth_threeraces_blackafricanamerican_nativehawaiianpacificislander_someotherrace,raceeth_threeraces_americanindianalaskanative_asian_nativehawaiianpacificislander,raceeth_threeraces_americanindianalaskanative_asian_someotherrace,raceeth_threeraces_americanindianalaskanative_nativehawaiianpacificislander_someotherrace,raceeth_threeraces_asian_nativehawaiianpacificislander_someotherrace,raceeth_fourraces_white_blackafricanamerican_americanindianalaskanative_asian,raceeth_fourraces_white_blackafricanamerican_americanindianalaskanative_nativehawaiianpacificislander,raceeth_fourraces_white_blackafricanamerican_americanindianalaskanative_someotherrace,raceeth_fourraces_white_blackafricanamerican_asian_nativehawaiianpacificislander,raceeth_fourraces_white_blackafricanamerican_nativehawaiianpacificislander_someotherrace,raceeth_fourraces_white_americanindianalaskanative_asian_nativehawaiianpacificislander,raceeth_fourraces_white_americanindianalaskanative_asian_someotherrace,raceeth_fourraces_white_americanindianalaskanative_nativehawaiianpacificislander_someotherrace,raceeth_fourraces_white_asian_nativehawaiianpacificislander_someotherrace,raceeth_fourraces_blackafricanamerican_americanindianalaskanative_asian_nativehawaiianpacificislander,raceeth_fourraces_blackafricanamerican_americanindianalaskanative_asian_someotherrace,raceeth_fourraces_blackafricanamerican_americanindianalaskanative_nativehawaiianpacificislander_someotherrace,raceeth_fourraces_blackafricanamerican_asian_nativehawaiianpacificislander_someotherrace,raceeth_fourraces_americanindianalaskanative_asian_nativehawaiianpacificislander_someotherrace,raceeth_fiveraces,raceeth_sixraces
0,109625.0,2453.0,6326.0,534.0,347.0,104.0,2801.0,926.0,341.0,6873.0,2801.0,926.0,341.0,6873.0,261.0,1380.0,210.0,3407.0,1046.0,777.0,132.0,140.0,49.0,250.0,166.0,67.0,32.0,30.0,12.0,6.0,47.0,466.0,82.0,770.0,79.0,12.0,112.0,83.0,16.0,36.0,25.0,6.0,8.0,16.0,8.0,250.0,20.0


In [76]:
moredata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 47 columns):
 #   Column                                                                                                         Non-Null Count  Dtype  
---  ------                                                                                                         --------------  -----  
 0   raceeth_tworaces_white_americanindianalaskanative                                                              1 non-null      float64
 1   raceeth_tworaces_white_nativehawaiianpacificislander                                                           1 non-null      float64
 2   raceeth_tworaces_blackafricanamerican_americanindianalaskanative                                               1 non-null      float64
 3   raceeth_tworaces_blackafricanamerican_nativehawaiianpacificislander                                            1 non-null      float64
 4   raceeth_tworaces_americanindianalaskanative_asian     

In [74]:
cols = moredata.columns

In [77]:
cols

Index(['raceeth_tworaces_white_americanindianalaskanative', 'raceeth_tworaces_white_nativehawaiianpacificislander',
       'raceeth_tworaces_blackafricanamerican_americanindianalaskanative', 'raceeth_tworaces_blackafricanamerican_nativehawaiianpacificislander',
       'raceeth_tworaces_americanindianalaskanative_asian', 'raceeth_tworaces_americanindianalaskanative_nativehawaiianpacificislander',
       'raceeth_tworaces_americanindianalaskanative_someotherrace', 'raceeth_tworaces_asiannativehawaiianpacificislander',
       'raceeth_tworaces_nativehawaiianotherpacificislandersomeotherrace',
       'raceeth_threeraces_white_blackafricanamerican_americanindianalaskanative', 'raceeth_tworaces_americanindianalaskanative_someotherrace',
       'raceeth_tworaces_asiannativehawaiianpacificislander', 'raceeth_tworaces_nativehawaiianotherpacificislandersomeotherrace',
       'raceeth_threeraces_white_blackafricanamerican_americanindianalaskanative',
       'raceeth_threeraces_white_blackafricana

In [78]:
moredata['all with some native'] = moredata[cols].sum(axis = 1)

In [80]:
somenative = moredata['all with some native'] 